Se carga el dataset que se utilizara

In [ ]:
from pandas import DataFrame
import pandas as pd

dataset =  DataFrame(pd.read_csv('windserie.csv', header=None))
print(dataset.head)

Se aplica media movil en los datos.

In [ ]:
from matplotlib import pyplot

pyplot.style.use("ggplot")
pyplot.plot(dataset[1:150])
pyplot.title('Datos Entrada')
pyplot.figure(num=None, figsize=(8, 6), dpi=320, facecolor='w', edgecolor='k')
pyplot.show()

dataset=dataset.rolling(window=8, min_periods=1).mean()

pyplot.plot(dataset[1:150])
pyplot.title('Media móvil Aplicada')
pyplot.figure(num=None, figsize=(8, 6), dpi=320, facecolor='w', edgecolor='k')
pyplot.show()

Se define el tamaño de ventana que tendra la serie de tiempos, y se transforma a una colección de datos para el entrenamiento supervisado del modelo.

In [ ]:
import Tools as tls

size = 5

dataset = tls.series_to_supervised(dataset,size, n_out=1, dropnan = True)
print(dataset)

Se normalizan los datos entre 0 y 1

In [ ]:
from sklearn import preprocessing

datasetnorm = preprocessing.minmax_scale(dataset, feature_range=(0, 1))
print(datasetnorm)
datasetnorm = DataFrame(datasetnorm)
print(datasetnorm.head)

Preparamos los datos para alimentar la red para entrenamiento y prueba.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

zy = datasetnorm.iloc[:, -1:]
zx = datasetnorm.iloc[:, :-1].values
X = np.array(zx, dtype='float64')
y = np.array(zy, dtype='float64')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=20)
print (X_train.shape, y_train.shape)

Se define nuestro modelo, con un numero de "vecinos" deseados a utilizarse, y se entrena

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=12)
knn.fit(X_train, y_train)

Realizamos predicción de valores de prueba, y comparamos con valores reales utilizando el error cuadratico medio.

In [ ]:
from sklearn.metrics import mean_squared_error

pred = knn.predict(X_test)
MSE = mean_squared_error(y_test, pred)

print ('MSE '+ str(MSE))

Se imprime grafica

In [ ]:
pyplot.style.use("ggplot")
pyplot.plot(y_test[1:100,]) #Valores ajustables para grafica
pyplot.plot(pred[1:100,])
pyplot.xlabel('Epochs')
pyplot.ylabel('Value')
pyplot.title('Predicción contra Real')
pyplot.figure(num=None, figsize=(8, 6), dpi=320, facecolor='w', edgecolor='k')
pyplot.show()

En este paso, realizaremos entrenamiento con multiples valores de vecinos, para buscar un parametro de vecinos que nos de resultado con mejor aptitud.

In [ ]:
neighbors = 150
MSEList = []
for k in range(1,neighbors):
    knn = KNeighborsRegressor(k)
    knn.fit(X_train, y_train)
    MSEList.append(mean_squared_error(y_test, knn.predict(X_test)))
    
pyplot.plot(range(1,neighbors), MSEList)
pyplot.xlabel('k-neighbors')
pyplot.ylabel('Mean Squared Error')
pyplot.title('Gráfica parametro k-Vecinos')
pyplot.figure(num=None, figsize=(8, 6), dpi=320, facecolor='w', edgecolor='k')
pyplot.show()